In [2]:
from __future__ import print_function
import cv2
import os
import glob
import numpy as np


import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
def load_data(categories):
    # Training and Testing
    data             = []
    target           = []
    size_image       = 32
    img_dir          = 'train'    
    
    for idx, category in enumerate(categories):
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)
        for f1 in files:
            img        = cv2.imread(f1)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            data.append(img_resize)
            target.append(idx)

    target     = keras.utils.to_categorical(target, len(categories))    
    data       = np.array(data)
    target     = np.array(target)
    stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)
    for test_idx, train_idx in stratSplit.split(data, target):        
        train_data   = data[train_idx]
        train_target = target[train_idx]                        
        test_data    = data[test_idx]
        test_target  = target[test_idx]
        
    # Validation
    val_data   = []
    val_target = []
    img_dir    = 'val'
    for idx, category in enumerate(categories):        
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)        
        for f1 in files:
            img        = cv2.imread(f1)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            val_data.append(img_resize)
            val_target.append(idx)
    
    val_target   = keras.utils.to_categorical(val_target, len(categories))                
    val_data     = np.array(val_data)
    val_target   = np.array(val_target)
    
    np.random.shuffle(val_data)
    np.random.shuffle(val_target)    
    
    return train_data, train_target, test_data, test_target, val_data, val_target

In [4]:
batch_size        = 32
num_classes       = 7
data_augmentation = True
num_predictions   = 20
save_dir          = os.path.join(os.getcwd(), 'saved_models')
model_name        = 'faces.h5'
categories  = ['angry','disgust','fear','happy','neutral','sad','surprise']
train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)

In [5]:
epochs            = 30

In [6]:
model = Sequential()
model.add(Conv2D(32, (4, 4), padding='same',input_shape=train_data.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(48, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(80, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Flatten())
# model.add(Dense(1024))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        1568      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 48)        24624     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 48)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 48)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 48)        0         
____________________

In [7]:

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

datagen.fit(train_data)

# Fit the model on the batches generated by datagen.flow().
generator = datagen.flow(train_data, train_target,batch_size=batch_size)
model.fit_generator(generator, epochs=epochs,validation_data=(test_data, test_target),workers=4, steps_per_epoch = len(generator))

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(test_data, test_target, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
73/73 [==============================] - 12s 166ms/step - loss: 12.0400 - acc: 0.2524 - val_loss: 11.9916 - val_acc: 0.2560
Epoch 2/30
73/73 [==============================] - 11s 157ms/step - loss: 11.9783 - acc: 0.2568 - val_loss: 11.9916 - val_acc: 0.2560
Epoch 3/30
73/73 [==============================] - 12s 163ms/step - loss: 11.9851 - acc: 0.2564 - val_loss: 11.9916 - val_acc: 0.2560
Epoch 4/30
73/73 [==============================] - 12s 166ms/step - loss: 11.9941 - acc: 0.2559 - val_loss: 11.9916 - val_acc: 0.2560
Epoch 5/30
73/73 [==============================] - 13s 178ms/step - loss: 11.9873 - acc: 0.2563 - val_loss: 11.9916 - val_acc: 0.2560
Epoch 6/30
73/73 [==============================] - 11s 152ms/step - loss: 11.9896 - acc: 0.2561 - val_loss: 11.9916 - val_acc: 0.2560
Epoch 7/30
73/73 [==============================] - 12s 162ms/step - loss: 11.9873 - acc: 0.2563

KeyboardInterrupt: 